In [ ]:
"""
Fashion-MNIST Classification (Modified to Dense Layers Only)

Autorzy:
- Kacper Sewruk s23466
- Michał Jastrzemski s26245

Ten notebook wykorzystuje zbiór Fashion-MNIST (wbudowany w Keras) i trenuje model MLP.
Zamiast używać warstw konwolucyjnych, spłaszczamy obrazy do wektora i używamy Dense.

Rysujemy również macierz pomyłek, a wyniki i logi zapisujemy w katalogu `logs/`.
"""

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix

os.makedirs('logs', exist_ok=True)

def load_fashion_mnist_data():
    """
    Wczytuje dane Fashion-MNIST z Keras.

    Zwraca
    -------
    X_train, X_test : ndarray
        Dane treningowe i testowe (28x28 obrazy w skali szarości).
    y_train, y_test : ndarray
        Etykiety treningowe i testowe (0-9).
    """
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    return X_train, X_test, y_train, y_test

def preprocess_data(X_train, X_test, y_train, y_test):
    """
    Przetwarza dane Fashion-MNIST:
    - Normalizuje piksele do [0,1].
    - One-hot encoding etykiet.
    - Spłaszcza obrazy (28x28 -> 784).

    Parametry
    ---------
    X_train, X_test : ndarray
        Dane treningowe i testowe.
    y_train, y_test : ndarray
        Etykiety w formie integer.

    Zwraca
    -------
    X_train, X_test, y_train_cat, y_test_cat : ndarray
        Przetworzone dane i etykiety one-hot.
    """
    X_train = X_train.astype('float32')/255.0
    X_test = X_test.astype('float32')/255.0

    # Spłaszczanie obrazów
    X_train = X_train.reshape(X_train.shape[0], -1)  # 28*28=784
    X_test = X_test.reshape(X_test.shape[0], -1)

    y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
    y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

    return X_train, X_test, y_train_cat, y_test_cat

def build_fashion_model(input_shape):
    """
    Buduje prosty model MLP dla Fashion-MNIST:
    - Kilka warstw Dense z aktywacją ReLU
    - Warstwa wyjściowa (10 klas, softmax)

    Parametry
    ---------
    input_shape : tuple
        Kształt wejścia, np. (784,)

    Zwraca
    -------
    model : tf.keras.Model
        Skompilowany model MLP.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate_model(X_train, X_test, y_train_cat, y_test_cat, y_test):
    """
    Trenuje i ocenia model MLP na Fashion-MNIST.
    Zapisuje:
    - logs/results_fashion_mnist.txt
    - logs/training_history_fashion_mnist.png
    Rysuje macierz pomyłek i zapisuje do logs/confusion_matrix.png.

    Parametry
    ---------
    X_train, X_test, y_train_cat, y_test_cat : ndarray
        Dane i etykiety one-hot do treningu i testów.
    y_test : ndarray
        Oryginalne etykiety (integer) potrzebne do confusion matrix.
    """
    model = build_fashion_model((X_train.shape[1],))
    history = model.fit(X_train, y_train_cat, validation_split=0.2, epochs=10, batch_size=64, verbose=0)
    loss, acc = model.evaluate(X_test, y_test_cat, verbose=0)

    with open('logs/results_fashion_mnist.txt', 'w') as f:
        f.write(f"Test Accuracy: {acc}\n")

    plt.plot(history.history['accuracy'], label='train acc')
    plt.plot(history.history['val_accuracy'], label='val acc')
    plt.legend()
    plt.title('Fashion-MNIST Training History (Dense Only)')
    plt.savefig('logs/training_history_fashion_mnist.png')
    plt.show()

    # Macierz pomyłek
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    cm = confusion_matrix(y_test, y_pred_classes)

    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Fashion-MNIST Confusion Matrix')
    plt.savefig('logs/confusion_matrix.png')
    plt.show()

def main():
    """
    Funkcja główna:
    1. Wczytuje dane Fashion-MNIST.
    2. Przetwarza dane (normalizacja, one-hot, spłaszczanie).
    3. Buduje i trenuje model MLP (Dense only).
    4. Rysuje macierz pomyłek i zapisuje logi.
    """
    X_train, X_test, y_train, y_test = load_fashion_mnist_data()
    X_train, X_test, y_train_cat, y_test_cat = preprocess_data(X_train, X_test, y_train, y_test)
    train_and_evaluate_model(X_train, X_test, y_train_cat, y_test_cat, y_test)

main()
